# Get members of groups based on Github repositories
Used for finding out exercise class membership 

In [ ]:
from github import Github # pip install PyGithub
import sys
import datetime
import pandas as pd
import numpy as np
import random
from collections import Counter

In [ ]:
# load personal access token
with open("C:/Users/gmf123/Dropbox/github_token.txt", mode = "r") as file:
    token = file.read()

year = "2023"    
class_name = "projects-" + year

### Load all repositories in this year's class room

In [ ]:
# a. access github through access token.
gh = Github(token)
org = gh.get_organization('NumEconCopenhagen')
all_repos = org.get_repos()

# b. locate all repositories in current class
disregard_repos = []

current_class = [repo.name for repo in all_repos if (class_name in repo.name) & (repo.name not in disregard_repos)]

In [ ]:
# see this years' repos
for r in current_class:
    print(r.removeprefix(class_name+"-"))

### Get members

In [ ]:
class_repos = []
groups = {}
admins = ["AndersMunkN", "JeppeDruedahl", "ChristianLangholzCarstensen"]
students = []

for repo in all_repos:

    if repo.name not in current_class: continue
    
    # get the set of collaborators for a repo - remove organization admins
    collaborators = repo.get_collaborators().get_page(0)
    members = [c.login for c in collaborators if c.login not in admins] 
    
    # add members to dictionary and to list of all students
    grp_name = repo.name.removeprefix(class_name+"-")  
    groups[grp_name] = members
    
    students.extend(members)


### To Excel

In [ ]:
gd = {k:pd.Series(v,dtype='str') for k,v in groups.items()}

groups_df = pd.DataFrame(gd) 
groups_df = groups_df.T
groups_df.rename(columns = {0:'Member1',1:'Member2',2:'Member3'}, inplace=True)
groups_df.fillna('',inplace=True)

groups_df.sort_index(inplace=True)

for col in ['Exercise class','Inaugural','Data','Model','Comments']:
    groups_df[col] = ''

In [ ]:
groups_df.reset_index(inplace=True)
groups_df.rename(columns={'index':'Group'}, inplace=True)
groups_df.head()

In [ ]:
groups_df.to_excel('Groups.xlsx',index=False)